<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers pandas torch

In [2]:
from google.colab import drive
import pandas as pd
from google.colab import drive
import shutil
from google.colab import drive
#import nltk

#Aquí seleccionamos el archivo desde nuestro equipo.

from google.colab import files
uploaded = files.upload()

Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [3]:
import pandas as pd

In [4]:
data = pd.read_excel('BASE_30122024.xlsx')

In [5]:
data = pd.DataFrame(data)

In [6]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoTokenizer, AutoModelWithLMHead

# Configuración para usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar los datos económicos
data

# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row.get('Año', 'NaN')}, Mes: {row.get('Mes', 'NaN')}, "
        f"Trimestre: {row.get('Trimestre', 'NaN')}, Frecuencia: {row.get('Frecuencia', 'NaN')}, "
        f"Año_comparación: {row.get('Año_comparación', 'NaN')}, País: {row.get('País', 'NaN')}, "
        f"Tendencia: {row.get('Tendencia', 'NaN')}, Sector: {row.get('Sector', 'NaN')}, "
        f"Indicador: {row.get('Indicador', 'NaN')}, Valor_actual: {row.get('Valor_actual', 'NaN')}, "
        f"Variación: {row.get('Variación', 'NaN')}, Valor_comparación: {row.get('Valor_comparación', 'NaN')}, "
        f"Entidad: {row.get('Entidad', 'NaN')}"
    )
    return texto_datos

# Crear una columna con los datos numéricos convertidos a texto
data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]


In [7]:
# Dataset personalizado
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        texto = self.data.iloc[idx]["texto_final"]
        encoding = self.tokenizer(
            texto,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        input_ids = encoding["input_ids"].flatten()
        attention_mask = encoding["attention_mask"].flatten()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,  # En GPT, las etiquetas son las mismas que la entrada
        }

In [8]:
# Cargar el tokenizador y modelo GPT-2 preentrenado
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1838: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [9]:
# Ajustar el tokenizador para manejar padding
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

# Crear el dataset y el DataLoader
dataset = NoticiasDataset(data, tokenizer)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./gpt2_noticias_model",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Entrenar el modelo
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,1.421600
200,0.588500
300,0.473200
400,0.442600
500,0.398100
600,0.376900
700,0.332400
800,0.347600
900,0.325700
1000,0.337800


TrainOutput(global_step=15940, training_loss=0.20525812094603457, metrics={'train_runtime': 2195.14, 'train_samples_per_second': 14.523, 'train_steps_per_second': 7.261, 'total_flos': 2082497495040000.0, 'train_loss': 0.20525812094603457, 'epoch': 10.0})

In [14]:
def generar_noticia(model, tokenizer, datos):
    entrada_texto = datos_a_texto(datos)
    inputs = tokenizer.encode(entrada_texto, return_tensors="pt").to(model.device)

    # Asegurar que el modelo tenga `pad_token_id` configurado correctamente
    if model.config.pad_token_id is None:
        model.config.pad_token_id = tokenizer.eos_token_id

    # Generar texto
    outputs = model.generate(
        inputs,
        max_length=128,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        attention_mask=torch.ones_like(inputs),  # Crear un attention_mask válido
        temperature=0.7,  # Control de la aleatoriedad (menor temperatura = menos aleatorio)
        top_p=0.9,  # Top-p sampling (para diversidad)
        do_sample=True  # Activa la aleatorización (en lugar de beam search)
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [16]:
# Ejemplo 1
new_data = {
    'Año': '2024',
    'Mes': 'enero',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '2023',
    'País': 'Colombia',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'Inflación',
    'Valor_actual': '9%',
    'Variación': '1%',
    'Valor_comparación': '8%',
    'Entidad': 'DANE',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: enero, Trimestre: , Frecuencia: Mensual, Año_comparación: 2023, País: Colombia, Tendencia: Crecimiento, Sector: Economía, Indicador: Inflación, Valor_actual: 9%, Variación: 1%, Valor_comparación: 8%, Entidad: DANE </s> Noticia: La tasa de inflación en Colombia se moderó al 9.2% en enero de2024. Este descenso se debe principalmente a la caída en los precios de alimentos y energía.  se beneficiarán principalmente con el aumento de las


In [20]:
# Ejemplo 1.1
new_data = {
    'Año': '2024',
    'Mes': 'enero',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '2023',
    'País': 'Colombia',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'Inflación',
    'Valor_actual': '9%',
    'Variación': '1%',
    'Valor_comparación': '8%',
    'Entidad': 'DANE',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: enero, Trimestre: , Frecuencia: Mensual, Año_comparación: 2023, País: Colombia, Tendencia: Crecimiento, Sector: Economía, Indicador: Inflación, Valor_actual: 9%, Variación: 1%, Valor_comparación: 8%, Entidad: DANE </s> Noticia: La tasa de inflación en Colombia se moderó al 9% en enero de2024. Este descenso fue impulsado principalmente por la caída en los precios de alimentos y energía..8% de las ventas externas de Colombia fueron de 8


In [21]:
# Ejemplo 1.2
new_data = {
    'Año': '2024',
    'Mes': 'enero',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '2023',
    'País': 'Colombia',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'Inflación',
    'Valor_actual': '9%',
    'Variación': '1%',
    'Valor_comparación': '8%',
    'Entidad': 'DANE',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: enero, Trimestre: , Frecuencia: Mensual, Año_comparación: 2023, País: Colombia, Tendencia: Crecimiento, Sector: Economía, Indicador: Inflación, Valor_actual: 9%, Variación: 1%, Valor_comparación: 8%, Entidad: DANE </s> Noticia: La tasa de inflación en Colombia se situó en 9.1% en enero de2024, una desaceleración en comparación con el 8.7% de 20.4% registrado en el mismo mes del año2023.


In [22]:
# Ejemplo 2
new_data = {
    'Año': '2024',
    'Mes': '',
    'Trimestre': 'Primer',
    'Frecuencia': 'Trimestral',
    'Año_comparación': '',
    'País': 'Argentina',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'PIB',
    'Valor_actual': '10%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: , Trimestre: Primer, Frecuencia: Trimestral, Año_comparación: , País: Argentina, Tendencia: Crecimiento, Sector: Economía, Indicador: PIB, Valor_actual: 10%, Variación: , Valor_comparación: , Entidad: </s> Noticia: La economía de Argentina creció a un ritmo anualizado del 10.3% en el primer trimestre de 2023, superando las expectativas del mercado.  Este crecimiento fue impulsado principalmente por el crecimiento de la actividad en los sectores de minería, comercio y servicios.


In [23]:
# Ejemplo 2.1
new_data = {
    'Año': '2024',
    'Mes': '',
    'Trimestre': 'Primer',
    'Frecuencia': 'Trimestral',
    'Año_comparación': '',
    'País': 'Argentina',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'PIB',
    'Valor_actual': '10%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: , Trimestre: Primer, Frecuencia: Trimestral, Año_comparación: , País: Argentina, Tendencia: Crecimiento, Sector: Economía, Indicador: PIB, Valor_actual: 10%, Variación: , Valor_comparación: , Entidad: </s> Noticia: El Producto Interno Bruto (PIB) de Argentina creció un 10% anual en el primer trimestre de 2023, superando las expectativas del mercado., impulsada por el crecimiento de la actividad en los sectores de comercio exterior y el incremento de las exportaciones por un


In [24]:
# Ejemplo 2.2
new_data = {
    'Año': '2024',
    'Mes': '',
    'Trimestre': 'Primer',
    'Frecuencia': 'Trimestral',
    'Año_comparación': '',
    'País': 'Argentina',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'PIB',
    'Valor_actual': '10%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: , Trimestre: Primer, Frecuencia: Trimestral, Año_comparación: , País: Argentina, Tendencia: Crecimiento, Sector: Economía, Indicador: PIB, Valor_actual: 10%, Variación: , Valor_comparación: , Entidad: </s> Noticia: La economía de Argentina creció a un ritmo anualizado del 10% en el primer trimestre de 2023, superando las expectativas del mercado. Este crecimiento fue impulsado principalmente por el crecimiento de la actividad en los sectores de comercio y la recuperación del consumo interno.


In [25]:
# Ejemplo 3
new_data = {
    'Año': '2020',
    'Mes': 'mayo',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Bolivia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Tasa_desempleo',
    'Valor_actual': '3%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2020, Mes: mayo, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Bolivia, Tendencia: Disminuye, Sector: Economía, Indicador: Tasa_desempleo, Valor_actual: 3%, Variación: , Valor_comparación: , Entidad: </s> Noticia: La tasa de desempleo en Bolivia descendió al 3.0% en mayo de 2020 tuvo una disminución de 0.8% puntos por encima de la tasa registrada en el mismo mes del año anterior, reflejando una tendencia a la baja del comercio exterior del


In [26]:
# Ejemplo 3.1
new_data = {
    'Año': '2020',
    'Mes': 'mayo',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Bolivia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Tasa_desempleo',
    'Valor_actual': '3%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2020, Mes: mayo, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Bolivia, Tendencia: Disminuye, Sector: Economía, Indicador: Tasa_desempleo, Valor_actual: 3%, Variación: , Valor_comparación: , Entidad: </s> Noticia: La tasa de desempleo en Bolivia baja al 3% en mayo de 2020 se ubica en 3.2% lo que indica una desaceleración en comparación con el mismo mes del año anterior. cae al 10.3% registrado en


In [27]:
# Ejemplo 3.2
new_data = {
    'Año': '2020',
    'Mes': 'mayo',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Bolivia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Tasa_desempleo',
    'Valor_actual': '3%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2020, Mes: mayo, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Bolivia, Tendencia: Disminuye, Sector: Economía, Indicador: Tasa_desempleo, Valor_actual: 3%, Variación: , Valor_comparación: , Entidad: </s> Noticia: La tasa de desempleo en Bolivia se disminuyó al 3.0% en mayo de 2020 tuvo una disminución de 0.8% puntos por encima de la tasa registrada en el mismo mes del año anterior. Este descenso se debe principalmente a la caída en


In [32]:
# Ejemplo 4
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Nicaragua',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Importaciones',
    'Valor_actual': '12000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Nicaragua, Tendencia: Disminuye, Sector: Economía, Indicador: Importaciones, Valor_actual: 12000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia: Las importaciones en Nicaragua cayeron un 1.3% en 1993 respecto al mismo mes del año 1992, reflejando una tendencia a la baja del comercio exterior del país. se registró una reducción en los precios en comparación con el del mismo período de 1992 IPC


In [34]:
# Ejemplo 4.1
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Nicaragua',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Importaciones',
    'Valor_actual': '12000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Nicaragua, Tendencia: Disminuye, Sector: Economía, Indicador: Importaciones, Valor_actual: 12000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia: Las importaciones en Nicaragua cayeron un 10% en 1993 respecto al mismo año 1992izaron 12.0 millones de dólares nan, se presentan en el informe sobre el comercio exterior de Nicaragua se proyecta que se ubique en 13.1, se espera que el


In [37]:
# Ejemplo 4.2
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Nicaragua',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Importaciones',
    'Valor_actual': '12000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Nicaragua, Tendencia: Disminuye, Sector: Economía, Indicador: Importaciones, Valor_actual: 12000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia, Nicaragua experimentó una disminución de las importaciones del 10.1% en 1993 respecto al año 1992 respecto a el mismo mes del año anterior. nan, se ubique en 11174.0, de acuerdo con la información de exportaciones del gobierno de Nicaragua.


In [39]:
# Ejemplo 5
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Uruguay',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Exportaciones',
    'Valor_actual': '10000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Uruguay, Tendencia: Disminuye, Sector: Economía, Indicador: Exportaciones, Valor_actual: 10000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia: En 1993 las exportaciones en Uruguay sumaron10000.0, registando así una caída del 0.1% respecto al año 1992 tuvo una disminución de 2.5%..6% en comparación con el mismo mes de 1992 se ubicó en los 100


In [42]:
# Ejemplo 5.1
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Uruguay',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Exportaciones',
    'Valor_actual': '10000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Uruguay, Tendencia: Disminuye, Sector: Economía, Indicador: Exportaciones, Valor_actual: 10000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia: Las exportaciones en Uruguay cayeron un 10% en 1993, un descenso en comparación con el 20% registrado en el mismo mes de 1992 Boliviana se presentó una reducción en la producción de crudo por parte de los países de la OPEP+ y las


In [44]:
# Ejemplo 5.2
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Uruguay',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Exportaciones',
    'Valor_actual': '10000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Uruguay, Tendencia: Disminuye, Sector: Economía, Indicador: Exportaciones, Valor_actual: 10000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia: Las exportaciones en Uruguay cayeron un 10% en 1993, reflejando una tendencia a la baja en el comercio exterior del país. se ubica en 10.6% sobre el nivel de ingresos de las exportaciones de baja por parte de los países de la OP


In [45]:
# Ejemplo 6
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Chile',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Precio_petróleo',
    'Valor_actual': '81',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Chile, Tendencia: Disminuye, Sector: Economía, Indicador: Precio_petróleo, Valor_actual: 81, Variación: , Valor_comparación: , Entidad: </s> Noticia: El precio del petróleo en Chile muestra una tendencia hacia la baja, en parte debido a factores como la menor demanda global y la transición hacia energías renovables, que afecta el mercado de combustibles fósiles y el comercio exterior de Chile. Las tensiones geopolíticas en Ucrania


In [46]:
# Ejemplo 6.1
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Chile',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Precio_petróleo',
    'Valor_actual': '81',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Chile, Tendencia: Disminuye, Sector: Economía, Indicador: Precio_petróleo, Valor_actual: 81, Variación: , Valor_comparación: , Entidad: </s> Noticia: En noviembre de 2015 se espera que el precio del petróleo en Chile llegue a los 81 dólares por barril, un aumento del 11% en comparación con el mes anterior., la cual se presentó en el informe de balanza comercial, mostró una reducción al


In [47]:
# Ejemplo 6.2
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Chile',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Precio_petróleo',
    'Valor_actual': '81',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Chile, Tendencia: Disminuye, Sector: Economía, Indicador: Precio_petróleo, Valor_actual: 81, Variación: , Valor_comparación: , Entidad: </s> Noticia: En noviembre de 2015 el precio del petróleo en Chile muestra una tendencia hacia la baja, en parte debido a factores como la menor demanda global y la transición hacia energías renovables, que afecta el mercado de combustibles fósiles. Este es un descenso en comparación con el 11


In [53]:
# Ejemplo 7
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Italia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Inversión_extranjera_directa',
    'Valor_actual': '19000',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Italia, Tendencia: Disminuye, Sector: Economía, Indicador: Inversión_extranjera_directa, Valor_actual: 19000, Variación: , Valor_comparación: , Entidad: </s> Noticia: La IED en Italia alcanzó los 190000 millones de dólares en noviembre de 2015 con una variación de 0.4% por debajo de la tasa registrada en el mismo mes del año 2023. Este es un descenso en comparación con


In [56]:
# Ejemplo 7.1
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Italia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Inversión_extranjera_directa',
    'Valor_actual': '19000',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Italia, Tendencia: Disminuye, Sector: Economía, Indicador: Inversión_extranjera_directa, Valor_actual: 19000, Variación: , Valor_comparación: , Entidad: </s> Noticia: La inversión extranjera directa (IED) en Italia alcanzó los 190000 millones de dólares en noviembre de 2015 lo que representa un crecimiento del 10% en comparación con el mismo mes del año anterior. reflejando una tendencia a la


In [57]:
# Ejemplo 7.2
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Italia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Inversión_extranjera_directa',
    'Valor_actual': '19000',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Italia, Tendencia: Disminuye, Sector: Economía, Indicador: Inversión_extranjera_directa, Valor_actual: 19000, Variación: , Valor_comparación: , Entidad: </s> Noticia: La inversión extranjera directa (IED) en Italia disminuyó un 20% en noviembre de 2015 hasta los 19200 millones de dólares. se presentó una reducción en el nivel de ingresos de inversión privada se recuperará en octubre de
